In [1]:
using Plots;gr()
using Optim
using DataFrames;using CSV
using Pkg
using LinearAlgebra;
using JuliaZH;JuliaZH.set_mirror("PKU")#JuliaZH.generate_startup("BFSU")
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)
using Base.Threads;nthreads()
using Meteor.ExactDiag;
using JLD2;using FileIO
using DifferentialEquations

In [13]:
function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3,:)
end
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end
    
function coherence_state(alpha)
    m=exp(alpha*a₊(dim)-alpha'*a₋(dim))*n0(dim)
    return  m*m'
end
    
function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end
    
function equ_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    for i in 1:n
        c[i,i]=1/n    
    end
    return c
end
    
dim=4
η=0.084 
    
function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));
a=a₊(dim)+a₋(dim)    
# a2=kron([0 0;1 0],exp(im*η*a))
# a3=kron([0 1;0 0],exp(-im*η*a))
a2=kron([0 0;1 0],im*η*a₊(dim))
a3=kron([0 1;0 0],-im*η*a₋(dim))   
#经过尝试发现,符合脉冲并不精准,都保留下来就很不精确
    
σ₊=[0 0;1 0]
σ₋=[0 1;0 0]
σ_x=[0 1;1 0]
σ_y=[0 -1im;1im 0]
σ_z=[1 0;0 -1]

function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,Int(sqrt(size(vec)[1])),:)
end
    
function temperature(nbar)
    return (6.626*1e-34*1e6*1)/(1.38*1e-23*log(1/nbar+1))
end
    
n=kron(I(2),a₊(dim)*a₋(dim))
en=kron([0 0;0 1],I(dim))

ν=1.
    
function H(Ω,Δ)
    unitary::Matrix{ComplexF64}=kron(I(dim*2),-im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3))+kron(transpose(im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3)),I(dim*2))
    return unitary
end
    
function Hv(Ω,Δ)
    unitary::Matrix{ComplexF64}=ν*n-Δ*en+Ω/2*a2+Ω'/2*a3
    return unitary
end


Hv (generic function with 1 method)

In [14]:
instate=kron([1. 0;0 0],equ_state(dim,2));

In [15]:
function R(t,phi,omega,delta)
    return exp(-im*t*Hv(omega*exp(im*phi),delta))
end

R (generic function with 1 method)

In [16]:
function swap_prod(dims,n)#蓝边带交换
    s=zeros(dim*2,dim*2)+I(dim*2)
    s[n+1,n+1]=0.;
    s[n+2+dims,n+2+dims]=0.;
    s[n+2+dims,n+1]=1.;
    s[n+1,n+2+dims]=1.;
    
    
    return s
end
swap=swap_prod(dim,0)

8×8 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [28]:
function feval(x)
    swapfit=R(x[3],x[5],0.1,1)*R(x[2],x[4],0.1,1)*R(x[1],0,0.1,1)
    opres=norm(norm.(swapfit)-norm.(swap))
    println(round.(x,digits=5),"    ",round(opres,digits=4))
    return opres
end

feval (generic function with 1 method)

In [43]:
# xini=[1.39359/η/0.1 1.41803/η/0.1 2.92217/η/0.1 1.34343  5.46806];
xini=[470.00471, 1262.15604, 1247.92811, 4.53369, 6.77461] #from0

lower=[0.,0,0,0,0];upper=[4*pi/η/0.1,4*pi/η/0.1,4*pi/η/0.1,2*pi,2*pi];
# res=optimize(feval,xini,ParticleSwarm(; lower = lower,upper = upper,n_particles = 10),Optim.Options(iterations = 10000))
# res=optimize(feval,xini,LBFGS())
    

In [44]:
xini

5-element Vector{Float64}:
  470.00471
 1262.15604
 1247.92811
    4.53369
    6.77461

In [45]:
res.minimizer

5-element Vector{Float64}:
  470.0047135133139
 1262.1560435737874
 1247.9281060929895
    4.533694799879266
    6.774609711963202

In [46]:
res.minimum

0.12451489627799936

In [47]:
function feval2(x)
    swapfit=R(x[3],x[5],0.1,1)*R(x[2],x[4],0.1,1)*R(x[1],0,0.1,1)
    
    return swapfit
end

feval2 (generic function with 1 method)

In [48]:
round.(DataFrame(abs.(feval2(xini)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0604,0.0,0.0,0.0,0.0,0.9982,0.0,0.0
2,0.0,0.9993,0.0,0.0,0.0,0.0,0.0367,0.0
3,0.0,0.0,0.9986,0.0,0.0,0.0,0.0,0.0525
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.9982,0.0,0.0,0.0,0.0,0.0604,0.0,0.0
7,0.0,0.0367,0.0,0.0,0.0,0.0,0.9993,0.0
8,0.0,0.0,0.0525,0.0,0.0,0.0,0.0,0.9986


In [49]:
# round.(DataFrame(feval2(xini)/(feval2(xini)[1]/abs.(feval2(xini))[1]),:auto),digits=4)

In [50]:
round.(DataFrame(abs.(feval2(res.minimizer)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0604,0.0,0.0,0.0,0.0,0.9982,0.0,0.0
2,0.0,0.9993,0.0,0.0,0.0,0.0,0.0367,0.0
3,0.0,0.0,0.9986,0.0,0.0,0.0,0.0,0.0525
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.9982,0.0,0.0,0.0,0.0,0.0604,0.0,0.0
7,0.0,0.0367,0.0,0.0,0.0,0.0,0.9993,0.0
8,0.0,0.0,0.0525,0.0,0.0,0.0,0.0,0.9986
